In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Masking
from keras.layers import Attention, LayerNormalization, Dropout
from keras.optimizers import Adam
import os
import tensorflow as tf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

# Read the Parquet file from Google Drive
parquet_path = "/content/drive/My Drive/dataset.parquet"
df = pd.read_parquet(parquet_path)

# Save as CSV
csv_output_path = "/content/drive/My Drive/Sinhala_english.csv"
df.to_csv(csv_output_path, index=False)

In [5]:
os.chdir("/content/drive/My Drive/Colab_Notebooks/")
os.getcwd()

'/content/drive/My Drive/Colab_Notebooks'

In [6]:
df=pd.read_csv('output.csv')

In [7]:
df

,english,sinhala
0,thank you for your understanding,තේරුම් ගත්තාට ස්තූතියි
1,There's not much time left for me now.,මට දැන් වැඩි වෙලාවක් ඉතිරි වෙලා නෑ.
2,I have been diagnosed with cancer.,මට පිළිකාවක් හැදිලා තියෙනවා.
3,An unsolved case from a year ago.,අවුරුද්දකට කලින් විසඳපු නැති කේස් එකක්
4,Okay? Come on.,හරි ? කමෝන්.
...,...,...
130,are you crazy man?,ඔහෙට පිස්සු හැදිලද මිනිහෝ?
131,"Good product, Have a very good quality, 100% G...","හොඳ නිෂ්පාදනයක්, ගොඩක් හොඳ ගුණාත්මක බවක් තියෙන..."
132,how to create drum,බෙරයක් හදන්නේ කොහොමද
133,"i'm ready,always",මම හැමතිස්සේම ලෑස්තියි


In [8]:
spa_sentences = df["sinhala"] + "\t" + df["english"]

# Save the concatenated sentences to a text file
with open("spa.txt", "w", encoding="utf-8") as file:
    for sentence in spa_sentences:
        file.write(str(sentence) + "\n")

In [9]:
text_file ="spa.txt"

In [10]:
text_file ="spa.txt"
with open(text_file) as f:
  lines=f.read().split("\n")[:-1]

In [11]:
i=0
for line in lines:
  print(line)
  i=i+1
  if(i==20):
    break

තේරුම් ගත්තාට ස්තූතියි	 thank you for your understanding
මට දැන් වැඩි වෙලාවක් ඉතිරි වෙලා නෑ.	There's not much time left for me now.
මට පිළිකාවක් හැදිලා තියෙනවා.	I have been diagnosed with cancer.
අවුරුද්දකට කලින් විසඳපු නැති කේස් එකක්	An unsolved case from a year ago.
හරි ? කමෝන්.	Okay? Come on.
හරි ? කමෝන්.	Okay? Come on.
හරි ? කමෝන්.	Okay? Come on.
අපි ගොඩක් මහන්සි වෙලා වැඩ කලොත් ඒක අපේ කාර්‍ය සාධනයට බලපානවා.	It'll affect our performance if we are too persistent.
ඔයා කවදාවත් හිතුවද ඒක කොච්චර හදකම්පා කරන දෙයක්ද කියලා	Have you ever thought about how heartbreaking it is
ඔයාට අපිව ආරක්ෂා කරන්න බැරි නම් එහෙනම් කොහේද සාධාරණය ?	If you can't protect us, then where's the justice?
හුවැන්ග් මින්ග් චෙන්ග් ඇත්තටම මැරිලද?	Is Huang Ming-Cheng really dead?
කොහොමද ඔයා මට මෙහෙම කරේ?	How could you do this to me?
කෝ මගේ දුව ?	Where's my daughter?
ඔයාගෙ සැමියා මගේ දුවව පැහැරගත්තා	Your husband kidnapped my daughter.
ඔයාගෙ සැමියා මගේ දුවව පැහැරගත්තා	Your husband kidnapped my daughter.
 මගේ දුව කොහෙද ?	Wher

In [12]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

ඇව්නි, ඇයි ඔයා ඔතන හිටගෙන මෙහෙට එන්න	Avni, why are you standing there come in.
මේ තමයි මම ඔයාට දෙන සප්රයිස් එක	This is my surprise for you.
ඔතන නවත්තන්න	-  Stop right there.
ඔතන නවත්තන්න	-  Stop right there.
හායි	hi
ඔහෙට පිස්සු හැදිලද මිනිහෝ?	are you crazy man?
හොඳ නිෂ්පාදනයක්, ගොඩක් හොඳ ගුණාත්මක බවක් තියෙනවා, 100% හොඳ වැඩ කරන තත්වයේ තියෙනවා	Good product, Have a very good quality, 100% Good working condition
බෙරයක් හදන්නේ කොහොමද	how to create drum
මම හැමතිස්සේම ලෑස්තියි 	i'm ready,always
nan


In [13]:
import random
import string
import re

text_pairs = []


for line in lines:
    if '\t' in line:
        english, sinhala = line.split("\t")
        sinhala = "[start]" + sinhala.strip() + "[end]"
        text_pairs.append((english.strip(), sinhala))
for i in range(3):
    print(random.choice(text_pairs))


('මම මේ වෙනකම් ඉවසුවේ ඔයාට දැනගන්න දෙන්න.', '[start]I endured it until now to let you know[end]')
('මේ චෙන් අක්කේ, කරුණාකරලා නවතින්න.', '[start]Sis Mei-Chen, please stop.[end]')
('මගේ හිතට එන පළවෙනි දේ', '[start]the first thing that appears in my mind[end]')


In [14]:
import random
random.shuffle(text_pairs)

In [15]:
num_val_samples =int(0.15*len(text_pairs))
num_train_samples=len(text_pairs) -2* num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [16]:
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 133
Training set size: 95
Validation set size: 19
Testing set size: 19


In [17]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

133

In [18]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [19]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [20]:
from tensorflow import keras
from tensorflow.keras import layers
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
    lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]

In [21]:
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [25]:
import tensorflow as tf

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "sinhala": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

# Assuming `train_pairs` and `val_pairs` are defined somewhere in your notebook
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

# Check a single batch
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")

# Safely checking and printing the 33rd batch
batch_list = list(train_ds.as_numpy_iterator())
batch_index = 32  # Since indexing starts at 0, 32 is the 33rd element

if len(batch_list) > batch_index:
    print(batch_list[batch_index])
else:
    print(f"Batch index {batch_index + 1} is out of range. The dataset contains only {len(batch_list)} batches.")


inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
Batch index 33 is out of range. The dataset contains only 2 batches.


In [24]:
batch_size=64

def format_dataset(eng,spa):
  eng=source_vectorization(eng)
  spa=target_vectorization(spa)
  return({
      "english":eng,
      "sinhala":spa[:,:-1],
  },spa[:,1:])

def make_dataset(pairs):
  eng_texts, spa_texts =zip(*pairs)
  eng_texts =list(eng_texts)
  spa_texts =list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape:{inputs['english'].shape}")
  print(f"inputs['sinhala'].shape:{inputs['sinhala'].shape}")
  print(f"targets.shape:{targets.shape}")

  inputs['english'].shape: (64, 20)
  inputs['sinhala'].shape: (64, 20)
  targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[32])

inputs['english'].shape:(64, 20)
inputs['sinhala'].shape:(64, 20)
targets.shape:(64, 20)


IndexError: list index out of range

In [26]:
import string
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [27]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [28]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask
        )
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


In [29]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
            })
        return config

In [30]:
embed_dim = 256
dense_dim = 2048
num_heads = 8


# Define the encoder inputs
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="english")

# Add positional embedding to the encoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Encode the inputs using TransformerEncoder
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# Define the decoder inputs
decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="sinhala")

# Add positional embedding to the decoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Decode the inputs using TransformerDecoder
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

# Apply dropout
x = layers.Dropout(0.5)(x)

# Generate decoder outputs
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Create the transformer model
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

In [31]:
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
2/2 [==============================] - 12s 4s/step - loss: 9.3831 - accuracy: 0.1167 - val_loss: 8.7813 - val_accuracy: 0.1667
Epoch 2/30
2/2 [==============================] - 5s 3s/step - loss: 8.6086 - accuracy: 0.1741 - val_loss: 8.3185 - val_accuracy: 0.1667
Epoch 3/30
2/2 [==============================] - 5s 3s/step - loss: 7.9707 - accuracy: 0.1741 - val_loss: 7.8670 - val_accuracy: 0.1667
Epoch 4/30
2/2 [==============================] - 6s 4s/step - loss: 7.4194 - accuracy: 0.1741 - val_loss: 7.5279 - val_accuracy: 0.1754
Epoch 5/30
2/2 [==============================] - 5s 4s/step - loss: 6.9243 - accuracy: 0.1759 - val_loss: 7.1808 - val_accuracy: 0.1842
Epoch 6/30
2/2 [==============================] - 6s 4s/step - loss: 6.4485 - accuracy: 0.1778 - val_loss: 6.8652 - val_accuracy: 0.1667
Epoch 7/30
2/2 [==============================] - 6s 4s/step - loss: 6.0938 - accuracy: 0.1759 - val_loss: 6.6669 - val_accuracy: 0.1667
Epoch 8/30
2/2 [========================

In [32]:
import numpy as np

# Define the vocabulary and index lookup for Spanish

spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))


-
ඔයාට ඒක අමුතුයි කියලා හිතෙන්නේ නැද්ද?
[start] stop right there[end]             in  to  the
-
මගේ දුව ඩූ-ඩූ තමයි මට ඉතිරි වෙලා තියෙන්නේ.
[start] stop right there[end]             in  to  the
-
ඔයාට බෑ ඔයාගේ දුවව ආපහු ජීවිතේට ගේන්න.
[start] stop right there[end]             in  to  the
-
ඔයා පෙන්ග්-පෙන්ග් අතුරුදහන් උන එකට ඔයාටම දොස් කියාගත්තා
[start] stop right there[end]             in  to  the
-
ඒත් මම විශ්වාස කරනවා
[start] i am not coming[end]            in  to  
-
පෙන්ග් පෙන්ග්
[start] stop right there[end]               to  
-
පෙන්ග් පෙන්ග්
[start] stop right there[end]               to  
-
පෙන්ග් පෙන්ග්
[start] stop right there[end]               to  
-
ඒත් මම විශ්වාස කරනවා
[start] i am not coming[end]            in  to  
-
ඒත් හුවාන්ග් මින්ග් චෙන්ග් විතරයි දන්නේ
[start] i am not coming[end]            in  to  the
-
ඒ ඔයා නිසා තමයි
[start] stop right there[end]             in  to  
-
වැරදියට පරිවර්තනය වෙලාද?  දැන්ම අපිට වාර්තා කරන්න
[start] stop right there[end]             in  

In [33]:
import numpy as np

# Define the vocabulary and index lookup for Sinhala
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print("English: ", input_sentence)
    print("Sinhala: ", decode_sequence(input_sentence))


-
English:  ඒත් හුවාන්ග් මින්ග් චෙන්ග් විතරයි දන්නේ
Sinhala:  [start] i am not coming[end]            in  to  the
-
English:  ඒත් හුවාන්ග් මින්ග් චෙන්ග් විතරයි දන්නේ
Sinhala:  [start] i am not coming[end]            in  to  the
-
English:  ඒත් මම විශ්වාස කරනවා
Sinhala:  [start] i am not coming[end]            in  to  
-
English:  මට ඔයාගෙ උපන්දින උත්සවයට එන්න පුළුවන් වුන එක ගැන සතුටුයි ඒක අමතක වෙන්නැති අත්දැකීමක්
Sinhala:  [start] stop right there[end]             in  to  the
-
English:  පෙන්ග් පෙන්ග්
Sinhala:  [start] stop right there[end]               to  
-
English:  ඔයා පෙන්ග්-පෙන්ග් අතුරුදහන් උන එකට ඔයාටම දොස් කියාගත්තා
Sinhala:  [start] stop right there[end]             in  to  the
-
English:  මට ඔයාගෙ උපන්දින උත්සවයට එන්න පුළුවන් වුන එක ගැන සතුටුයි ඒක අමතක වෙන්නැති අත්දැකීමක්
Sinhala:  [start] stop right there[end]             in  to  the
-
English:  පෙන්ග් පෙන්ග්
Sinhala:  [start] stop right there[end]               to  
-
English:  ඔයා පෙන්ග්-පෙන්ග් අතුරුදහන් උන එකට ඔයාටම දොස

In [40]:
import numpy as np

# Define the vocabulary and index lookup for Sinhala
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

# Get English input from the user
input_sentence = input("Enter an English sentence: ")

# Translate the input sentence to Sinhala
translated_sentence = decode_sequence(input_sentence)

# Print the translated sentence
print("Sinhala translation:", translated_sentence)



Enter an English sentence: Nice to meet you
Sinhala translation: [start] stop right there[end]             in  to  
